In [ ]:
# DATA PREPARATION
# --------------------------------------------------------------------------------------

# Select data: Determine which data sets will be used and document reasons for 
# inclusion/exclusion.
# 
# Clean data: Often this is the lengthiest task. Without it, you’ll likely fall victim
# to garbage-in, garbage-out. A common practice during this task is to correct, impute,
# or remove erroneous values.
# 
# Construct data: Derive new attributes that will be helpful. For example, derive 
# someone’s body mass index from height and weight fields.
# 
# Integrate data: Create new data sets by combining data from multiple sources.
# 
# Format data: Re-format data as necessary. For example, you might convert string values
# that store numbers to numeric values so that you can perform mathematical operations.

In [1]:
import os

import pandas as pd
from ydata_profiling import ProfileReport

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

os.makedirs(f"../output/c-data-preparation", exist_ok=True)
os.makedirs(f"../data/c-data-preparation", exist_ok=True)

YEAR = "2022"

In [2]:
# Read data

file = f"../data/b-data-understanding/{YEAR}-grouped-by-occurrence.csv"

df = pd.read_csv(file)

described = df.describe(include="all")
described.loc["nan"] = df.isnull().mean() * 100

print(described)

                   id data_inversa dia_semana   horario     uf            br            km municipio                            causa_acidente              tipo_acidente classificacao_acidente   fase_dia sentido_via condicao_metereologica tipo_pista tracado_via uso_solo       pessoas        mortos  feridos_leves  feridos_graves        ilesos     ignorados       feridos      veiculos      latitude     longitude regional delegacia             uop
count    19895.000000        19895      19895     19895  19895  19847.000000  19847.000000     19895                                     19895                      19895                  19895      19895       19895                  19895      19895       19895    19895  19895.000000  19895.000000   19895.000000    19895.000000  19895.000000  19895.000000  19895.000000  19895.000000  19895.000000  19895.000000    19895     19895           19884
unique            NaN          365          7       978      3           NaN           NaN       448  

In [ ]:
# Select features
# --------------------------------------------------------------------------------------

# Only some features will be used for clustering. However, other features are also
# interesting for further analysis, when the clusters are known. Therefore, we will keep
# all of them in the dataset.

In [ ]:
# Gather additional data
# --------------------------------------------------------------------------------------

In [3]:
# Clean data
# --------------------------------------------------------------------------------------

# 1. Drop nans

n1 = len(df)
df = df.dropna()
n2 = len(df)

df = df.reset_index(drop=True)

# TODO: impute missing values
# 
# Only 3 columns with missing values: br, km, uop
# They can be imputed by considering lat, long, and city features

print(f"Number of rows dropped: {n1 - n2}")

# 2. Drop useless columns

df = df.drop(columns=["id"])

# 3. Format text data

# TODO: don't think it's necessary, text looks consistent (e.g. no typos)
# 
# str: lower case
# str: remove punctuation
# str: remove special characters
# str: remove accents
# str: remove extra spaces

Number of rows dropped: 48


In [4]:
# Feature engineering
# --------------------------------------------------------------------------------------

df.data_inversa = pd.to_datetime(df.data_inversa, format="%Y-%m-%d").dt.date
df.horario = pd.to_datetime(df.horario, format="%H:%M:%S").dt.time

func = lambda row: pd.Timestamp.combine(row.data_inversa, row.horario)
df["timestamp"] = df.apply(func, axis=1)

df["ano"] = df.timestamp.dt.year
df["mes"] = df.timestamp.dt.month
df["dia"] = df.timestamp.dt.day
# Already have "dia_semana" column
# df["dia_semana"] = df.data_inversa.dt.dayofweek

df["hora"] = df.timestamp.dt.hour
df["minuto"] = df.timestamp.dt.minute

df = df.drop(columns=["data_inversa", "horario", "timestamp"])

In [5]:
# Data type conversion
# --------------------------------------------------------------------------------------

dtypes = {
    # "id": "Int64",
    # "data_inversa": "datetime64[ns]",
    "dia_semana": "category",
    # "horario": "datetime64[ns]",
    "uf": "category",
    "br": "category",
    "km": "float64",
    "municipio": "category",
    "causa_acidente": "category",
    "tipo_acidente": "category",
    "classificacao_acidente": "category",
    "fase_dia": "category",
    "sentido_via": "category",
    "condicao_metereologica": "category",
    "tipo_pista": "category",
    "tracado_via": "category",
    "uso_solo": "category",
    "pessoas": "Int64",
    "mortos": "Int64",
    "feridos_leves": "Int64",
    "feridos_graves": "Int64",
    "ilesos": "Int64",
    "ignorados": "Int64",
    "feridos": "Int64",
    "veiculos": "Int64",
    "latitude": "float64",
    "longitude": "float64",
    "regional": "category",
    "delegacia": "category",
    "uop": "category",
    "ano": "Int64",
    "dia": "Int64",
    "mes": "Int64",
    "hora": "Int64",
    "minuto": "Int64",
}

df = df.astype(dtypes)

# Save non-preprocessed data
dst = f"../data/c-data-preparation/{YEAR}-raw.csv"
df.to_csv(dst, index=False)

In [6]:
# Encoding and normalization
# --------------------------------------------------------------------------------------

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Check if all columns have allowed data types
allowed_dtypes = ["category", "Int64", "int64", "float64"]
msg = "Invalid data type. Allowed data types: category, Int64, float64"
assert all(df[col].dtype.name in allowed_dtypes for col in df.columns), msg

# NOTE: no need to save label encoder or scaler, since after clustering we add a new
# "cluster" column to the non-preprocessed data 

# 1. Encode categorical columns

for column in df.select_dtypes(include=["category"]):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

# 2. Normalize all columns

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df.head()

,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,hora,minuto
0,0.833333,0.0,0.037037,0.045144,0.152466,0.582090,1.000000,0.0,1.0,1.0,0.750,0.0,0.000000,0.0,0.023810,0.142857,0.033333,0.000000,0.000000,0.000000,0.030303,0.0,0.786093,0.962317,0.0,0.000000,0.303030,0.0,0.0,0.000000,0.086957,0.677966
1,0.833333,1.0,0.148148,0.109576,0.363229,0.119403,0.266667,0.0,1.0,0.0,0.125,1.0,0.333333,0.0,0.071429,0.142857,0.033333,0.166667,0.000000,0.142857,0.060606,0.1,0.645287,0.440311,1.0,0.714286,0.767677,0.0,0.0,0.000000,0.347826,0.084746
2,0.833333,1.0,0.000000,0.191108,0.060538,0.208955,0.533333,0.5,1.0,0.0,0.125,1.0,0.333333,0.0,0.071429,0.000000,0.033333,0.000000,0.071429,0.000000,0.030303,0.2,0.610510,0.989759,1.0,0.392857,0.686869,0.0,0.0,0.000000,0.521739,0.338983
3,0.000000,0.0,0.037037,0.011491,0.152466,0.194030,0.933333,0.5,1.0,1.0,0.750,0.0,0.000000,0.0,0.023810,0.000000,0.066667,0.000000,0.000000,0.000000,0.060606,0.0,0.789134,0.987151,0.0,0.000000,0.303030,0.0,0.0,0.033333,0.304348,0.169492
4,0.000000,1.0,0.814815,0.279207,0.670404,0.985075,0.400000,0.5,1.0,0.0,0.125,1.0,0.000000,0.0,0.071429,0.000000,0.100000,0.000000,0.023810,0.000000,0.090909,0.1,0.586717,0.814873,1.0,0.500000,0.717172,0.0,0.0,0.033333,0.347826,0.338983


In [7]:
described = df.describe(include="all")
described.loc["nan"] = df.isnull().mean() * 100

described

,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,hora,minuto
count,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.000000,19847.0,19847.000000,19847.000000,19847.000000,19847.000000
mean,0.494500,0.505265,0.308047,0.300728,0.474484,0.479385,0.548590,0.571724,0.791505,0.463647,0.289187,0.529954,0.415909,0.468635,0.032609,0.009033,0.027751,0.042937,0.025319,0.021831,0.033035,0.069023,0.616098,0.758403,0.507104,0.360126,0.395255,0.0,0.506713,0.487787,0.557649,0.415571
std,0.335779,0.433744,0.292614,0.240932,0.276192,0.339551,0.288847,0.240788,0.268943,0.498689,0.296805,0.471870,0.223414,0.499028,0.039948,0.041659,0.031639,0.094170,0.032671,0.060772,0.031243,0.073012,0.183048,0.208209,0.433558,0.285696,0.285591,0.0,0.312482,0.291543,0.272933,0.296497
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.166667,0.000000,0.037037,0.125855,0.253363,0.149254,0.333333,0.500000,0.666667,0.000000,0.125000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.491064,0.616057,0.000000,0.071429,0.131313,0.0,0.272727,0.233333,0.347826,0.169492
50%,0.500000,0.500000,0.222222,0.236662,0.459641,0.552239,0.533333,0.500000,1.000000,0.000000,0.125000,0.500000,0.555556,0.000000,0.023810,0.000000,0.033333,0.000000,0.023810,0.000000,0.030303,0.100000,0.623385,0.796305,0.500000,0.321429,0.343434,0.0,0.545455,0.500000,0.608696,0.423729
75%,0.833333,1.000000,0.555556,0.428181,0.708520,0.820896,0.933333,0.500000,1.000000,1.000000,0.625000,1.000000,0.555556,1.000000,0.047619,0.000000,0.033333,0.000000,0.023810,0.000000,0.030303,0.100000,0.750871,0.945774,1.000000,0.642857,0.646465,0.0,0.818182,0.733333,0.782609,0.677966
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000
nan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [8]:
dst = f"../data/c-data-preparation/{YEAR}-preprocessed.csv"
df.to_csv(dst, index=False)

In [9]:
# Profile report

profile = ProfileReport(df, title="Profiling Report")
profile.to_file(f"../output/c-data-preparation/{YEAR}-profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]